In [64]:
# imports (not subject to tariffs)

import pandas as pd
import math


In [93]:
# Getting and cleaning the data.

file_names = ["target_prices_combined_east_south_central",
              "target_prices_combined_mountain",
              "target_prices_combined_pacific",
              "target_prices_combined_south_atlantic",
              "target_prices_combined_west_north_central",
              "target_prices_combined_west_south_central",
              "target_prices_NE",
              "target_prices_NEC_full",
              "target_pricing_MA"]

for file in file_names:
    globals()[file] = pd.read_csv("scraped data to combine into full dataframe/" + file + ".csv")

    
# This is the section where I clean up some of the data so I can mash it all together in a sec.

# Starting with defining some functions that I'll use to clean the data, of course.

# Some zips aren't 5 numbers long.
def zip_extender_and_stringifier(code):
    if len(str(int(code))) != 5:
        return "0" + str(int(code))
    return str(int(code))

def price_floater(price):
    # I did not need to make this one line. I probably should not have made this one line. But I did anyways, because it made me feel awesome.
    return float(price.split('$')[-1].split('.')[0]) + float(price.split('$')[-1].split('.')[1])/100

def airpod_capitalizer(product):
    if product == 'Airpods':
        return 'AirPods'
    return product

def classification_fixer(classification):
    return classification.title().split(' ')[0]


target_prices_NE = target_prices_NE.drop("Unnamed: 5", axis=1)  # Drops mysterious sixth column.
target_prices_NE = target_prices_NE.dropna()  # Removes null values from data.
target_prices_NE["Zip Code"] = target_prices_NE["Zip Code"].apply(zip_extender_and_stringifier)  # Fixes zips being registered as floats
target_prices_NE["Price"] = target_prices_NE["Price"].apply(price_floater)  # Turns prices into floats
target_prices_NE["Classification"] = target_prices_NE["Classification"].apply(classification_fixer)  # Fixes capitalization on classifications
target_prices_NE = target_prices_NE.rename(columns={'Classification': 'Area Type'})  # ...only to immediately rename the column.

# Doing the same with the other two. I tried to do this as a loop, but pandas didn't like that very much.
target_prices_NEC_full = target_prices_NEC_full.dropna()
target_prices_NEC_full["Zip Code"] = target_prices_NEC_full["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_NEC_full["Price"] = target_prices_NEC_full["Price"].apply(price_floater)
target_prices_NEC_full["Classification"] = target_prices_NEC_full["Classification"].apply(classification_fixer)
target_prices_NEC_full["Product"] = target_prices_NEC_full["Product"].apply(airpod_capitalizer) # This one changes Airpods to AirPods, to match the others' capitalization of the word. 
target_prices_NEC_full = target_prices_NEC_full.rename(columns={'Classification': 'Area Type'})

target_pricing_MA = target_pricing_MA.dropna()
target_pricing_MA["Zip Code"] = target_pricing_MA["Zip Code"].apply(zip_extender_and_stringifier)
target_pricing_MA["Price"] = target_pricing_MA["Price"].apply(price_floater)
target_pricing_MA["Classification"] = target_pricing_MA["Classification"].apply(classification_fixer)
target_pricing_MA = target_pricing_MA.rename(columns={'Classification': 'Area Type'})


# I messed up some of my data collection, so I'm fixing that manually(ish) for the files I know are affected.

target_prices_combined_mountain = target_prices_combined_mountain.drop(range(0,275,25))
target_prices_combined_mountain = target_prices_combined_mountain.drop(range(10,275,25))  # I accidentally duplicated 59718. I'm gonna have to re-scrape mountain later. fml.

target_prices_combined_pacific = target_prices_combined_pacific.drop(range(0,165,15))

target_prices_combined_west_south_central = target_prices_combined_west_south_central.drop(range(0,143,13))


# I have no idea why my code is breaking. i'm just gonna try this out of desperation.
target_prices_combined_east_south_central["Zip Code"] = target_prices_combined_east_south_central["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_combined_mountain["Zip Code"] = target_prices_combined_mountain["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_combined_pacific["Zip Code"] = target_prices_combined_pacific["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_combined_south_atlantic["Zip Code"] = target_prices_combined_south_atlantic["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_combined_west_north_central["Zip Code"] = target_prices_combined_west_north_central["Zip Code"].apply(zip_extender_and_stringifier)
target_prices_combined_west_south_central["Zip Code"] = target_prices_combined_west_south_central["Zip Code"].apply(zip_extender_and_stringifier)
# Oh hey, this actually fixed it! Well I'm keeping these here because they make all the column additions good.



# And now I get to pivot and combine them. This part's really satisfying.

files_to_concat = []

for file in file_names:
    df = globals()[file]
    
    print("pivoting", file)
    product_prices = df.pivot(index='Zip Code', columns='Product', values='Price')
    
    print("location+class for", file)
    location_and_class = df.drop_duplicates("Zip Code")
    location_and_class = location_and_class.drop(['Product', 'Price'], axis=1)
    try:
        location_and_class = location_and_class.drop('Category', axis=1)
    except:
        pass
    location_and_class = location_and_class.set_index('Zip Code')

    print("combining", file)
    out4 = pd.concat([product_prices, location_and_class], axis=1)
    #display(out4)
    print("done!")
    
    globals()["pivot_" + file] = out4  # I don't know if I'll need this? but I'll keep it in, just in case people want individual files.
    files_to_concat.append(globals()["pivot_" + file])  # This adds it to the list of frames to concat together.

full_data = pd.concat(files_to_concat)
display(full_data)



pivoting target_prices_combined_east_south_central
location+class for target_prices_combined_east_south_central
combining target_prices_combined_east_south_central
done!
pivoting target_prices_combined_mountain
location+class for target_prices_combined_mountain
combining target_prices_combined_mountain
done!
pivoting target_prices_combined_pacific
location+class for target_prices_combined_pacific
combining target_prices_combined_pacific
done!
pivoting target_prices_combined_south_atlantic
location+class for target_prices_combined_south_atlantic
combining target_prices_combined_south_atlantic
done!
pivoting target_prices_combined_west_north_central
location+class for target_prices_combined_west_north_central
combining target_prices_combined_west_north_central
done!
pivoting target_prices_combined_west_south_central
location+class for target_prices_combined_west_south_central
combining target_prices_combined_west_south_central
done!
pivoting target_prices_NE
location+class for target_pri

,AirPods,Bananas,Cotton Swabs,Diapers,Eggs,Gum,Milk,Shampoo,T-Shirt,Toilet Paper,Vegetable Oil,Location,Area Type
Zip Code,,,,,,,,,,,,,
35226,179.99,1.99,3.39,44.99,4.99,1.79,2.99,9.99,9.6,12.99,9.39,"Birmingham, AL",Urban
35601,179.99,1.99,3.39,44.99,4.99,1.79,2.99,9.99,9.6,12.99,9.39,"Decatur, AL",Suburban
36526,179.99,1.99,3.39,44.99,4.99,1.79,3.59,9.99,9.6,12.99,9.39,"Daphne, AL",Rural
37801,179.99,1.99,3.39,44.99,4.99,1.79,4.69,9.99,9.6,12.99,9.39,"Maryville, TN",Rural
38017,179.99,1.99,3.29,44.99,4.99,1.79,4.69,9.99,9.6,12.99,9.39,"Collierville, TN",Suburban
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16601,179.99,1.99,3.39,44.99,4.99,1.69,5.19,9.99,NaN,12.99,9.39,"201 W Beaver Ave,, State College, Pennsylvania...",Rural
16803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6,NaN,NaN,"315 Colonnade Blvd,",Rural
17013,179.99,1.99,3.39,44.99,4.99,1.69,4.69,9.99,9.6,12.99,9.39,"246 Westminster Dr,, Carlisle, Pennsylvania 17...",Suburban


In [8]:
# This data is found from this website: https://warehouse.ninja/target-distribution-center-locations/
# I inputted it manually, unfortunately.

warehouse_data = [[34.6449932755897,-86.8473292167007],
                  [33.44847758394,-112.216516477379],
                  [35.4443517679111,-119.184883986437],
                  [34.1129032695058,-117.481059984653],
                  [34.1556649469121,-117.430952966951],
                  [33.8829312577458,-117.266820757673],
                  [38.6830449523077,-121.721678642205],
                  [34.1342886776624,-117.416355370662],
                  [38.2782732480079,-104.467643668806],
                  [39.8102931481305,-104.986460657673],
                  [30.2323049532679,-82.6359163730191],
                  [31.7765737205048,-81.3526562288365],
                  [31.7776443503854,-81.3539269910744],
                  [31.4215451402305,-83.5182432359064],
                  [32.1739955290448,-81.1755252030619],
                  [21.4792398115305,-158.016842329534],
                  [41.9176226241567,-88.7369320728707],
                  [41.8303900458523,-87.7238539644176],
                  [41.8002418685309,-87.7811810344034],
                  [41.4744170168815,-88.1627609565227],
                  [41.9218023710286,-87.9598046121732],
                  [39.7475498809776,-86.2948367214139],
                  [42.4760732243742,-92.468984016849],
                  [38.9662278505392,-95.6943807344034],
                  [38.8653486027871,-76.7299089768994],
                  [44.9647972364202,-85.2029499259567],
                  [45.0979857630195,-93.2491908864199],
                  [44.9933873277923,-93.2149055171079],
                  [44.9452591409893,-92.9121249623863],
                  [40.5251569148739,-74.2592140725181],
                  [40.089765436428,-74.8200981032102],
                  [39.7817015818632,-75.3450861543144],
                  [43.178721716844,-73.7161892865095],
                  [42.9386799514717,-74.2384685711634],
                  [35.6225966445846,-81.3098264595263],
                  [39.9520393926857,-83.3482983306921],
                  [39.9471432044632,-83.3518352558173],
                  [44.5794790802794,-123.115068328716],
                  [40.1989563009857,-76.8052668801052],
                  [39.8890231958012,-77.6729397],
                  [34.1794119094042,-80.6818378840045],
                  [32.4347483116907,-97.0555079725181],
                  [34.5570298314302,-97.1446958584889],
                  [32.8696311238109,-95.6389149051154],
                  [32.8807284643133,-96.901032357673],
                  [29.9461573473188,-95.5288642134904],
                  [38.0306730381623,-78.998851715346],
                  [36.7076274133422,-76.6651214220913],
                  [47.0821524927726,-122.771626284653],
                  [43.080037381646,-88.4760084558174],
                  [42.4825228934735,-92.4709803136662]]


In [94]:
# This data is from this website: https://gist.github.com/pramodpendyala/e5688b6a63d2983eac804bbaa1fd7cc0

address_data = pd.read_csv('2023_Gaz_zcta_national.txt', sep="\t", header=None)
address_data.columns = address_data.iloc[0]
address_data = address_data.drop(0)
# Why is the column name like this. Who is responsible. I just want to talk.
address_data.rename(columns={'INTPTLONG                                                                                                                                  ':'INTPTLONG'}, inplace=True)

address_data['INTPTLAT'] = address_data['INTPTLAT'].apply(lambda x: float(x))
address_data['INTPTLONG'] = address_data['INTPTLONG'].apply(lambda x: float(x))

address_data = address_data.drop(["ALAND", "AWATER", "ALAND_SQMI", "AWATER_SQMI"], axis=1)

address_data = address_data.rename(columns={'GEOID': 'Zip Code'})
address_data = address_data.set_index('Zip Code')



full_data = full_data.join(address_data, how='left')

def closest_to_distro_center(row):
    distances = []
    
    
    for coordinate in warehouse_data:
        # I'll be honest, I have no idea how this works. I don't know most of my trig knowledge anymore, and I am trusting this stackoverflow person with my life.
        # https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
        earth_radius = 3959  # This is earth's radius in miles. I can switch this to fm by changing the number to 6371
        
        distance_lat = math.radians(row["INTPTLAT"] - coordinate[0])
        distance_lon = math.radians(row["INTPTLONG"] - coordinate[1])
        
        lat1 = math.radians(row["INTPTLAT"])
        lat2 = math.radians(coordinate[0])
        
        a = pow(math.sin(distance_lat / 2), 2) + (pow(math.sin(distance_lon / 2), 2) * math.cos(lat1) * math.cos(lat2))
        
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distances.append(earth_radius * c)
        
    
    return min(distances)


full_data["Closest Distro Center"] = full_data.apply(closest_to_distro_center, axis=1)

full_data = full_data.drop(["INTPTLAT", "INTPTLONG"], axis=1)
display(full_data)


# Ok now to add the rent data to the columns.

rent = pd.read_csv("scraped data to combine into full dataframe/df_rent.csv")
#display(rent)
rent = rent.drop(["Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12", "Unnamed: 13", "Unnamed: 14", "Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19"], axis=1)
#display(rent)
rent = rent.dropna()
rent = rent.rename(columns={'ZIP': 'Zip Code'})
rent["Zip Code"] = rent["Zip Code"].apply(zip_extender_and_stringifier)  # Fixes zips being registered as floats
rent = rent.set_index('Zip Code')
#display(rent)

full_data = full_data.join(rent, how='left')
display(full_data)



full_data.to_csv("full_data.csv", index=False)









,AirPods,Bananas,Cotton Swabs,Diapers,Eggs,Gum,Milk,Shampoo,T-Shirt,Toilet Paper,Vegetable Oil,Location,Area Type,Closest Distro Center
Zip Code,,,,,,,,,,,,,,
01040,179.99,1.99,3.39,44.99,4.99,1.79,4.19,9.99,9.6,12.99,9.39,"Bangor, ME 04401-3629",Rural,86.245840
02114,179.99,1.99,3.39,44.99,5.49,1.69,4.19,9.99,9.6,12.99,9.39,"100 Cambridge St, Ste 2,, Boston, Massachusett...",Urban,145.695293
02122,179.99,1.99,3.39,44.99,5.49,1.79,4.19,9.99,9.6,12.99,9.39,"500 Geneva Ave,, Dorchester, Massachusetts 021...",Suburban,148.321890
02865,179.99,1.99,3.39,44.99,4.99,1.69,3.89,9.99,9.6,12.99,9.39,"371 Putnam Pike Ste A-250,\nSmithfield, Rhode ...",Urban,144.662267
02886,NaN,1.99,NaN,44.99,4.99,1.79,NaN,NaN,9.6,NaN,9.39,"400 Bald Hill Rd, Warwick, RI 02886-6101",Suburban,153.968645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98614,179.99,1.99,3.39,44.99,4.49,1.79,3.69,9.99,9.6,12.99,9.39,"Chinook, WA",Urban,77.515806
98626,179.99,1.99,3.39,44.99,4.49,1.79,3.69,9.99,9.6,12.99,9.39,"Keslo, WA",Rural,62.878349
98901,179.99,1.99,3.39,44.99,1.99,1.79,3.59,9.99,9.6,12.99,9.39,"Yakima, WA",Suburban,118.479044


,AirPods,Bananas,Cotton Swabs,Diapers,Eggs,Gum,Milk,Shampoo,T-Shirt,Toilet Paper,...,Area Type,Closest Distro Center,Unemployment Rate,Studio,1BR,2BR,3BR,4BR,Division,Region
Zip Code,,,,,,,,,,,,,,,,,,,,,
01040,179.99,1.99,3.39,44.99,4.99,1.79,4.19,9.99,9.6,12.99,...,Rural,86.245840,3.9,"$1,044","$1,205","$1,496","$1,823","$2,037",New England,Northeast
02114,179.99,1.99,3.39,44.99,5.49,1.69,4.19,9.99,9.6,12.99,...,Urban,145.695293,3.0,"$2,163","$2,288","$2,711","$3,266","$3,594",New England,Northeast
02122,179.99,1.99,3.39,44.99,5.49,1.79,4.19,9.99,9.6,12.99,...,Suburban,148.321890,3.4,"$2,163","$2,288","$2,711","$3,266","$3,594",New England,Northeast
02865,179.99,1.99,3.39,44.99,4.99,1.69,3.89,9.99,9.6,12.99,...,Urban,144.662267,3.7,"1,233","$1,319","$1,614","$1,945","$2,359",New England,Northeast
02886,NaN,1.99,NaN,44.99,4.99,1.79,NaN,NaN,9.6,NaN,...,Suburban,153.968645,2.8,"1,233","$1,319","$1,614","$1,945","$2,359",New England,Northeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98614,179.99,1.99,3.39,44.99,4.49,1.79,3.69,9.99,9.6,12.99,...,Urban,77.515806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98626,179.99,1.99,3.39,44.99,4.49,1.79,3.69,9.99,9.6,12.99,...,Rural,62.878349,7.1,$963,"$1,186","$1,404","$1,967","$2,358",Pacific,West
98901,179.99,1.99,3.39,44.99,1.99,1.79,3.59,9.99,9.6,12.99,...,Suburban,118.479044,6.7,$903,"$1,004","$1,317","$1,845","$2,212",Pacific,West
